# Taller: Análisis de Instagram con Apify API + Gemini  
Autor: (tu nombre)  
Fecha de generación: 2025-05-21


**Objetivos**

1. Obtener datos públicos de Instagram usando el actor **`apify/instagram-api-scraper`**.  
2. Limpiar y pre‑procesar captions e información de posts.  
3. Clasificar sentimiento de captions con la API de Google Generative AI (Gemini).  
4. Extraer temas dominantes con LDA y nombrarlos con Gemini.  
5. Segmentar autores o posts según métricas de engagement.  
6. Diseñar una micro‑campaña basada en los insights descubiertos.


In [2]:
!wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/TercerCorte/instagram_posts.csv

--2025-12-04 03:48:56--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/TercerCorte/instagram_posts.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/TercerCorte/instagram_posts.csv [following]
--2025-12-04 03:48:56--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/TercerCorte/instagram_posts.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18361753 (18M) [text/plain]
Saving to: ‘instagram_posts.csv’

instagram_posts.csv 100%[===================>]  17.51M 

In [3]:
!pip install -qU requests pandas google-generativeai nltk seaborn wordcloud scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 86.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
import pandas as pd
df = pd.read_csv('instagram_posts.csv')
df.head()

,inputUrl,id,username,url,fullName,biography,externalUrls,externalUrl,externalUrlShimmed,followersCount,...,musicInfo.artist_name,musicInfo.song_name,musicInfo.uses_original_audio,musicInfo.should_mute_audio,musicInfo.should_mute_audio_reason,musicInfo.audio_id,locationName,locationId,paidPartnership,sponsors
0,https://www.instagram.com/f1/,3637646598679593202,f1,https://www.instagram.com/p/DJ7hE70tIDy/,FORMULA 1®,It's All To Drive For!\n \nThis story can only...,"[{'title': 'Overtake of the month vote', 'lynx...",https://f1.com/Overtake,https://l.instagram.com/?u=https%3A%2F%2Ff1.co...,34614194,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.instagram.com/f1/,3637494954817152582,f1,https://www.instagram.com/p/DJ6-mOeJuZG/,FORMULA 1®,It's All To Drive For!\n \nThis story can only...,"[{'title': 'Overtake of the month vote', 'lynx...",https://f1.com/Overtake,https://l.instagram.com/?u=https%3A%2F%2Ff1.co...,34614194,...,f1movie,Original audio,True,False,NaN,1.403914e+15,NaN,NaN,NaN,NaN
2,https://www.instagram.com/f1/,3637535605467005779,f1,https://www.instagram.com/p/DJ7H1xVskNT/,FORMULA 1®,It's All To Drive For!\n \nThis story can only...,"[{'title': 'Overtake of the month vote', 'lynx...",https://f1.com/Overtake,https://l.instagram.com/?u=https%3A%2F%2Ff1.co...,34614194,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.instagram.com/f1/,3635278159597499148,f1,https://www.instagram.com/p/DJzGjnCi3MM/,FORMULA 1®,It's All To Drive For!\n \nThis story can only...,"[{'title': 'Overtake of the month vote', 'lynx...",https://f1.com/Overtake,https://l.instagram.com/?u=https%3A%2F%2Ff1.co...,34614194,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.instagram.com/f1/,3637456706674459999,f1,https://www.instagram.com/p/DJ615pHMYVf/,FORMULA 1®,It's All To Drive For!\n \nThis story can only...,"[{'title': 'Overtake of the month vote', 'lynx...",https://f1.com/Overtake,https://l.instagram.com/?u=https%3A%2F%2Ff1.co...,34614194,...,NaN,NaN,NaN,NaN,NaN,NaN,Monaco,16366456.0,NaN,NaN



### 🔍 Preguntas – Sección 1 (Exploración)

1. ¿Cuántos posts hay en total?  
2. ¿Qué tipos de contenido (imagen, vídeo, carrusel) predominan?  
3. ¿Cuál es el rango de fechas cubierto por los posts?  
4. ¿Qué post obtuvo más 'likes' y cuál crees que es la razón?


In [5]:
total_posts = len(df)
print(f"Total number of posts: {total_posts}")

Total number of posts: 200


In [6]:
content_types = df['productType'].value_counts()
print("Content Type Distribution:")
print(content_types)

Content Type Distribution:
productType
clips    86
Name: count, dtype: int64


In [7]:
product_type_distribution = df['productType'].value_counts(dropna=False)
print("Full Content Type Distribution based on 'productType' column:")
print(product_type_distribution)

Full Content Type Distribution based on 'productType' column:
productType
NaN      114
clips     86
Name: count, dtype: int64


In [8]:
print(df.columns)

Index(['inputUrl', 'id', 'username', 'url', 'fullName', 'biography',
       'externalUrls', 'externalUrl', 'externalUrlShimmed', 'followersCount',
       'followsCount', 'hasChannel', 'highlightReelCount', 'isBusinessAccount',
       'joinedRecently', 'businessCategoryName', 'private', 'verified',
       'profilePicUrl', 'profilePicUrlHD', 'igtvVideoCount', 'relatedProfiles',
       'latestIgtvVideos', 'postsCount', 'fbid', 'type', 'shortCode',
       'caption', 'hashtags', 'mentions', 'commentsCount', 'firstComment',
       'latestComments', 'dimensionsHeight', 'dimensionsWidth', 'displayUrl',
       'images', 'alt', 'likesCount', 'timestamp', 'childPosts',
       'ownerFullName', 'ownerUsername', 'ownerId', 'isSponsored',
       'taggedUsers', 'coauthorProducers', 'isCommentsDisabled', 'videoUrl',
       'videoViewCount', 'videoPlayCount', 'productType', 'videoDuration',
       'isPinned', 'musicInfo.artist_name', 'musicInfo.song_name',
       'musicInfo.uses_original_audio', 'musicI

In [9]:
print(df['type'].value_counts(dropna=False))

type
Video      86
Sidecar    83
Image      31
Name: count, dtype: int64


In [10]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
min_date = df['timestamp'].min()
max_date = df['timestamp'].max()
print(f"Date Range: {min_date} to {max_date}")

Date Range: 2025-05-05 00:59:11+00:00 to 2025-05-21 21:01:28+00:00


In [11]:
most_liked_post = df.loc[df['likesCount'].idxmax()]

print("Post con más 'likes':\n")
print(f"ID del post: {most_liked_post['id']}")
print(f"URL del post: {most_liked_post['url']}")
print(f"Número de 'likes': {most_liked_post['likesCount']}")
print(f"Tipo de contenido: {most_liked_post['type']}")
print(f"Caption: {most_liked_post['caption']}")

Post con más 'likes':

ID del post: 3630879158915563525
URL del post: https://www.instagram.com/p/DJjeVuZiAAF/
Número de 'likes': 1927755
Tipo de contenido: Image
Caption: Let’s ride. 

#F1TheMovie only in theaters June 27. 

#F1 #Formula1



## 🧹 Sección 2 (Limpieza)

Explica por qué es importante limpiar y normalizar el texto de los captions.  
Enumera tres tipos de “ruido” que removerás (URLs, emojis, menciones, etc.) y da un ejemplo de cada uno.


In [12]:
import re, nltk, string
nltk.download('stopwords')
from nltk.corpus import stopwords

def clean_caption(text):
    text = text.lower()
    text = re.sub(r"http\S+", " ", text)        # URLs
    text = re.sub(r"@[\w_]+", " ", text)        # Menciones
    text = re.sub(r"[#]", " ", text)             # Hashtags (opcional: conservar)
    text = re.sub(r"[{}]".format(string.punctuation), " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['caption_clean'] = df['caption'].fillna("").apply(clean_caption)
df[['caption', 'caption_clean']].head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,caption,caption_clean
0,It certainly got the drivers’ seal of approval...,it certainly got the drivers’ seal of approval...
1,It’s go time! Get your tickets for #F1TheMovie...,it’s go time get your tickets for f1themovie n...
2,Rolling up to #F1TheMovie! 🤩 Our drivers have ...,rolling up to f1themovie 🤩 our drivers have ar...
3,A DOMINANT WIN FROM MAX VERSTAPPEN!! 👏\n\nThe ...,a dominant win from max verstappen 👏 the reign...
4,Monte Carlo ready 😍🇲🇨\n\n#McLaren #M7AReborn #F1,monte carlo ready 😍🇲🇨 mclaren m7areborn f1


1. Limpiar y normalizar el texto de los captions es importante porque mejora la calidad del análisis que se hará después, por ejemplo: extracción de temas, clasificación o modelos de lenguaje. los algoritmos pueden malinterpretar el contenido, producir resultados sesgados o perder información relevante.
2. emojis (Amamos esta nueva colección!!! 💕✨), enlaces (Compra aquí 👉 https://mipagina.com/productos) o menciones a usuarios (Gracias por la invitación @juan_perez 😄)


## 😊 Sección 3 (Sentimiento)

7. Presenta la distribución global de sentimientos y comenta.  
8. ¿Qué tipo de contenido genera mayor proporción de sentimientos positivos y negativos?  
9. Elige un pico de sentimiento negativo y analiza el contexto con un ejemplo de post.


7. La mayoría de las publicaciones de la muestra (8 de 10) se clasificaron como «positivas». Una proporción menor (2 de 10) se clasificaron como «neutras».No hubo publicaciones clasificadas como «negativas» en esta muestra en particular.

In [15]:
import os
import time
import pandas as pd

client = OpenAI()

def call_llm(prompt):

  response = client.responses.create(
  model="gpt-4o-mini",
  input=prompt
)
  return response.output_text

def classify_sentiment(text):
    prompt = (f"Clasifica el sentimiento del siguiente caption de Instagram como "
              f"'positivo', 'neutral' o 'negativo' siempre en minuscula. Solo responde con una palabra.\n\nCaption:\n{text}")
    time.sleep(2)
    return call_llm(prompt)


sample = df.sample(10, replace=True, random_state=42)
sample['sentimiento'] = sample['caption_clean'].apply(classify_sentiment)

sentiment_by_type = pd.crosstab(sample['type'], sample['sentimiento'], normalize='index')
print("Proportion of sentiments by content type:")
print(sentiment_by_type)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************THIA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}}

Publicaciones con imágenes: Todas las publicaciones con imágenes se clasificaron al 100% como positivas
Publicaciones con sidecar: todas las publicaciones con sidecar se clasificaron al 100% como positivas
Publicaciones con vídeos: Las publicaciones con vídeos mostraron una mezcla, con un 60 % clasificadas como positivas y un 40 % como neutras

In [ ]:
negative_posts = sample[sample['sentimiento'] == 'negativo']

if not negative_posts.empty:
    print("Example of a post with negative sentiment:")
    example_post = negative_posts.iloc[0]
    print(f"ID del post: {example_post['id']}")
    print(f"URL del post: {example_post['url']}")
    print(f"Tipo de contenido: {example_post['type']}")
    print(f"Caption: {example_post['caption']}")
    print(f"Sentimiento: {example_post['sentimiento']}")
else:
    print("No posts with 'negativo' sentiment were found in the sampled data.")

No posts with 'negativo' sentiment were found in the sampled data.


In [ ]:
sample['sentimiento'].value_counts()


,count
sentimiento,
positivo,8
neutral,2



## 🗂 Sección 4 (Temas)
9. Haga lo mismo con sentidimiento con tipo de contenido y tema
10. Lista los nombres de los temas generados. ¿Alguno es inesperado?  
11. Con un heatmap *tipo de contenido × tema*, indica qué tema es “propiedad” de cada formato.  
12. Para el tema dominante, proporciona dos insights accionables.


In [1]:
import time

def classify_sentiment(text):
    prompt = (f"Clasifica el tipo de contenido del siguiente caption de Instagram como "
              f"'resultados_estadisticas', 'entre_bastidores' o 'tecnologia_vehiculos' siempre en minuscula. Solo responde con una palabra.\n\nCaption:\n{text}")
    time.sleep(2)  # Evitar rate limits
    return call_llm(prompt)

# ⚠️ Usa una muestra para no exceder la cuota de la API
sample = df.sample(10, replace = True ,random_state=42)
sample['topic'] = sample['caption_clean'].apply(classify_sentiment)

NameError: name 'df' is not defined

In [ ]:
sample['topic'].value_counts()

In [ ]:
pd.crosstab(sample['topic'], sample['type'])

In [ ]:
!pip install -qU requests pandas google-generativeai nltk seaborn wordcloud scikit-learn gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 81.8 MB/s eta 0:00:00


In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.models import LdaMulticore
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('stopwords')

print("NLTK wordnet and stopwords downloaded, and gensim libraries imported.")

NLTK wordnet and stopwords downloaded, and gensim libraries imported.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
sns.heatmap(pd.crosstab(sample['topic'], sample['type']))


## 👥 Sección 5 (Segmentación)

13. Describe cada cluster en una frase (actividad y tono).  
14. ¿Qué segmento priorizarías para viralizar un mensaje y por qué?  
15. Propón una acción de engagement distinta para cada segmento.


In [ ]:
call_llm("""Describe cada cluster en una frase (actividad y tono). damelo como una lista de python.topic entre_bastidores resultados_estadisticas
type
Image 1 1
Sidecar 1 2
Video 5 0
""")


## 📝 Sección 6 (Micro‑campaña)

16. Presenta tus tres captions generados.  
17. Justifica  
&nbsp;&nbsp;a) Tema elegido.  
&nbsp;&nbsp;b) Tono y horario óptimos.  
18. Define un KPI de éxito y la meta para la campaña.


In [ ]:
def generar_caption(topic, tone='inspirador'):
    prompt = (f"Actúa como community manager. Crea un caption de máximo 220 caracteres "
              f"sobre el tema '{topic}'. Tono {tone}. No incluyas hashtags ni menciones.")
    return call_llm(prompt)

for t in ['exploración espacial', 'innovación', 'ciudadanía global']:
    print('→', generar_caption(t))


→ La exploración espacial nos invita a soñar en grande y a desafiar límites. Cada estrella que vemos es un destino por descubrir y un recordatorio de que el universo guarda infinitas posibilidades. ¡Sigamos mirando hacia arriba!
→ La innovación es el motor del cambio. Cada idea audaz nos acerca a un futuro lleno de posibilidades. Atrévete a soñar, a crear y a transformar. ¡El mundo está esperando tu chispa!
→ La ciudadanía global es más que un concepto; es un compromiso. Al entendernos y apoyarnos unos a otros, construimos un mundo donde la diversidad brilla y la empatía guía nuestras acciones. ¡Juntos somos más fuertes!


In [ ]:
lista_tonos =  {"tipo": "Image", "actividad": "Imágenes visuales", "tono": "Informativo"},    {"tipo": "Sidecar", "actividad": "Contenido adicional interactivo", "tono": "Engaging"},  {"tipo": "Video", "actividad": "Videos explicativos", "tono": "Educativo"}

In [ ]:
def generar_caption(topic,tone):
    prompt = (f"Actúa como community manager. Crea un caption de máximo 220 caracteres "
              f"sobre el tema '{topic}'. Tono {tone}. No incluyas hashtags ni menciones.")
    return call_llm(prompt)

for lista in lista_tonos:
  print (generar_caption(lista['actividad'], lista['tono']))
  print ("_______________")


### 📦 Exporta y entrega

1. Ejecuta todo el notebook (**Runtime → Run all**).  
2. Descarga el notebook (File → Download → .ipynb) y el memo en *PDF* o *Markdown*.  
3. Sube ambos archivos a la plataforma de la clase antes de la fecha establecida.
